In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd

In [2]:
# Import the function from the .py script
from eia_api_fetcher import fetch_usa_data

# Call the function and get the DataFrame
df_oil = fetch_usa_data()
df_oil.head()

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units,country,state,year,month,days_in_month,daily_value
228,2000-01-01,SFL,FLORIDA,EPC0,Crude Oil,FPF,Field Production,MCRFPFL1,Florida Field Production of Crude Oil (Thousan...,360,MBBL,USA,Florida,2000,1,31,11.612903
229,2000-02-01,SFL,FLORIDA,EPC0,Crude Oil,FPF,Field Production,MCRFPFL1,Florida Field Production of Crude Oil (Thousan...,349,MBBL,USA,Florida,2000,2,29,12.034483
230,2000-03-01,SFL,FLORIDA,EPC0,Crude Oil,FPF,Field Production,MCRFPFL1,Florida Field Production of Crude Oil (Thousan...,373,MBBL,USA,Florida,2000,3,31,12.032258
231,2000-04-01,SFL,FLORIDA,EPC0,Crude Oil,FPF,Field Production,MCRFPFL1,Florida Field Production of Crude Oil (Thousan...,385,MBBL,USA,Florida,2000,4,30,12.833333
232,2000-05-01,SFL,FLORIDA,EPC0,Crude Oil,FPF,Field Production,MCRFPFL1,Florida Field Production of Crude Oil (Thousan...,411,MBBL,USA,Florida,2000,5,31,13.258065


In [3]:
df_oil.shape

(9966, 17)

In [4]:
#Inspect data for any missing values
df_oil.isna().sum()

period                0
duoarea               0
area-name             0
product               0
product-name          0
process               0
process-name          0
series                0
series-description    0
value                 0
units                 0
country               0
state                 0
year                  0
month                 0
days_in_month         0
daily_value           0
dtype: int64

In [5]:
df_oil['state'].nunique()

33

There are some states missing in the USA, let's import the geopandas spatial dataset to inspect our dataset.

In [6]:
states_prov = gpd.read_file('/Users/boluwatifeoduyemi/Documents/Data Science/Python Packages/ne_110m_admin_1_states_provinces/ne_110m_admin_1_states_provinces.shp')
states_prov.head()

,featurecla,scalerank,adm1_code,diss_me,iso_3166_2,wikipedia,iso_a2,adm0_sr,name,name_alt,...,FCLASS_ID,FCLASS_PL,FCLASS_GR,FCLASS_IT,FCLASS_NL,FCLASS_SE,FCLASS_BD,FCLASS_UA,FCLASS_TLC,geometry
0,Admin-1 scale rank,2,USA-3514,3514,US-MN,http://en.wikipedia.org/wiki/Minnesota,US,1,Minnesota,MN|Minn.,...,None,None,None,None,None,None,None,None,None,"POLYGON ((-89.95766 47.28691, -90.13175 47.292..."
1,Admin-1 scale rank,2,USA-3515,3515,US-MT,http://en.wikipedia.org/wiki/Montana,US,1,Montana,MT|Mont.,...,None,None,None,None,None,None,None,None,None,"POLYGON ((-116.04823 49.00037, -113.0595 49.00..."
2,Admin-1 scale rank,2,USA-3516,3516,US-ND,http://en.wikipedia.org/wiki/North_Dakota,US,1,North Dakota,ND|N.D.,...,None,None,None,None,None,None,None,None,None,"POLYGON ((-97.22894 49.00089, -97.21414 48.902..."
3,Admin-1 scale rank,2,USA-3517,3517,US-HI,http://en.wikipedia.org/wiki/Hawaii,US,8,Hawaii,HI|Hawaii,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-155.93665 19.05939, -155.9080..."
4,Admin-1 scale rank,2,USA-3518,3518,US-ID,http://en.wikipedia.org/wiki/Idaho,US,1,Idaho,ID|Idaho,...,None,None,None,None,None,None,None,None,None,"POLYGON ((-116.04823 49.00037, -115.9678 47.95..."


In [7]:
# Let's identify the missing states.
# Get the unique state names or codes from both datasets
oil_states = set(df_oil["state"].unique())  # States in df_oil
states_prov_states = set(states_prov["name"].unique())  # States in states_prov

# Find the states in states_prov but not in geo_maps_oil
missing_states = states_prov_states - oil_states

# Print the missing states
print("States in states_prov but missing in df_oil:", missing_states)

States in states_prov but missing in df_oil: {'New Jersey', 'District of Columbia', 'Wisconsin', 'Iowa', 'Washington', 'Vermont', 'North Carolina', 'Massachusetts', 'Maine', 'Idaho', 'Oregon', 'Hawaii', 'Connecticut', 'Georgia', 'Minnesota', 'South Carolina', 'New Hampshire', 'Maryland', 'Rhode Island', 'Delaware'}


In [8]:
# How many states are missing?
len(missing_states)

20

For each missing state, we will create a dataframe and assign dates just like the period column in df_oil

In [9]:
# Create a DataFrame with the first day of every month from 2000 to 2025
periods = pd.date_range(start="2000-01-01", end="2025-04-01", freq="MS")  # 'MS' means Month Start
all_periods_data = pd.DataFrame({
    'period': periods,
    'value': 0  # Assign 0 to all values
})

all_periods_data['period'] = pd.to_datetime(all_periods_data['period'], errors='coerce')

all_periods_data.head()

,period,value
0,2000-01-01,0
1,2000-02-01,0
2,2000-03-01,0
3,2000-04-01,0
4,2000-05-01,0


In [10]:
# Extract data for missing states from states_prov
missing_states_data = states_prov[states_prov["name"].isin(missing_states)][["name"]].copy()

# Assign values of 0 to the 'value' and 'daily_value' column for missing states
missing_states_data["value"] = 0
missing_states_data["daily_value"] = 0

In [11]:
# Add a key column for cross join
missing_states_data['key'] = 1
all_periods_data['key'] = 1

# Perform cross join to get all combinations
missing_states_new = pd.merge(missing_states_data, all_periods_data, on='key').drop('key', axis=1)
missing_states_new

,name,value_x,daily_value,period,value_y
0,Minnesota,0,0,2000-01-01,0
1,Minnesota,0,0,2000-02-01,0
2,Minnesota,0,0,2000-03-01,0
3,Minnesota,0,0,2000-04-01,0
4,Minnesota,0,0,2000-05-01,0
...,...,...,...,...,...
6075,Maine,0,0,2024-12-01,0
6076,Maine,0,0,2025-01-01,0
6077,Maine,0,0,2025-02-01,0
6078,Maine,0,0,2025-03-01,0


In [12]:
# Add the missing states new data to df_oil
df_oil = pd.concat([df_oil, missing_states_new], ignore_index=True)

# Extract the year and month from the 'period' column
df_oil['year'] = df_oil['period'].dt.year
df_oil['month'] = df_oil['period'].dt.month

# Verify the updated df_oil
df_oil

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units,country,state,year,month,days_in_month,daily_value,name,value_x,value_y
0,2000-01-01,SFL,FLORIDA,EPC0,Crude Oil,FPF,Field Production,MCRFPFL1,Florida Field Production of Crude Oil (Thousan...,360.0,MBBL,USA,Florida,2000,1,31.0,11.612903,NaN,NaN,NaN
1,2000-02-01,SFL,FLORIDA,EPC0,Crude Oil,FPF,Field Production,MCRFPFL1,Florida Field Production of Crude Oil (Thousan...,349.0,MBBL,USA,Florida,2000,2,29.0,12.034483,NaN,NaN,NaN
2,2000-03-01,SFL,FLORIDA,EPC0,Crude Oil,FPF,Field Production,MCRFPFL1,Florida Field Production of Crude Oil (Thousan...,373.0,MBBL,USA,Florida,2000,3,31.0,12.032258,NaN,NaN,NaN
3,2000-04-01,SFL,FLORIDA,EPC0,Crude Oil,FPF,Field Production,MCRFPFL1,Florida Field Production of Crude Oil (Thousan...,385.0,MBBL,USA,Florida,2000,4,30.0,12.833333,NaN,NaN,NaN
4,2000-05-01,SFL,FLORIDA,EPC0,Crude Oil,FPF,Field Production,MCRFPFL1,Florida Field Production of Crude Oil (Thousan...,411.0,MBBL,USA,Florida,2000,5,31.0,13.258065,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16041,2024-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024,12,NaN,0.000000,Maine,0.0,0.0
16042,2025-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,1,NaN,0.000000,Maine,0.0,0.0
16043,2025-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,2,NaN,0.000000,Maine,0.0,0.0
16044,2025-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,3,NaN,0.000000,Maine,0.0,0.0


In [13]:
# Now let's replace the NaN values in the state column with what is in the corresponding name column
df_oil['state'] = df_oil['state'].fillna(df_oil['name'])

# Let's also add the corresponding country i.e. USA to the NaN values in the country column 
df_oil['country'] = df_oil['country'].fillna('USA')

df_oil

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units,country,state,year,month,days_in_month,daily_value,name,value_x,value_y
0,2000-01-01,SFL,FLORIDA,EPC0,Crude Oil,FPF,Field Production,MCRFPFL1,Florida Field Production of Crude Oil (Thousan...,360.0,MBBL,USA,Florida,2000,1,31.0,11.612903,NaN,NaN,NaN
1,2000-02-01,SFL,FLORIDA,EPC0,Crude Oil,FPF,Field Production,MCRFPFL1,Florida Field Production of Crude Oil (Thousan...,349.0,MBBL,USA,Florida,2000,2,29.0,12.034483,NaN,NaN,NaN
2,2000-03-01,SFL,FLORIDA,EPC0,Crude Oil,FPF,Field Production,MCRFPFL1,Florida Field Production of Crude Oil (Thousan...,373.0,MBBL,USA,Florida,2000,3,31.0,12.032258,NaN,NaN,NaN
3,2000-04-01,SFL,FLORIDA,EPC0,Crude Oil,FPF,Field Production,MCRFPFL1,Florida Field Production of Crude Oil (Thousan...,385.0,MBBL,USA,Florida,2000,4,30.0,12.833333,NaN,NaN,NaN
4,2000-05-01,SFL,FLORIDA,EPC0,Crude Oil,FPF,Field Production,MCRFPFL1,Florida Field Production of Crude Oil (Thousan...,411.0,MBBL,USA,Florida,2000,5,31.0,13.258065,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16041,2024-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,USA,Maine,2024,12,NaN,0.000000,Maine,0.0,0.0
16042,2025-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,USA,Maine,2025,1,NaN,0.000000,Maine,0.0,0.0
16043,2025-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,USA,Maine,2025,2,NaN,0.000000,Maine,0.0,0.0
16044,2025-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,USA,Maine,2025,3,NaN,0.000000,Maine,0.0,0.0


In [14]:
# Import Canda Data

def load_canadian_data(path="estimated-production-canadian-crude-oil-equivalent.xlsx"):
    df_raw = pd.read_excel(path, sheet_name="HIST - barrels per day")
    df_raw.dropna(how='all', inplace=True)
    df_raw.dropna(axis=1, how='all', inplace=True)
    df_raw.rename(columns={df_raw.columns[0]: 'period'}, inplace=True)
    df_raw['period'] = pd.to_datetime(df_raw['period'], errors='coerce')
    df_raw.dropna(subset=['period'], inplace=True)
    exclude = ['Canada Total', 'Raw Mined Bitumen', 'Raw In Situ Bitumen']
    df_raw = df_raw.loc[:, ~df_raw.columns.isin(exclude)]
    province_groups = {}
    for col in df_raw.columns[1:]:
        if isinstance(col, str) and len(col) >= 2:
            prov = col.strip()[:2]
            province_groups.setdefault(prov, []).append(col)
    data = {'period': df_raw['period']}
    for prov, cols in province_groups.items():
        data[prov] = df_raw[cols].sum(axis=1) / 1000
    df_can = pd.DataFrame(data)
    df_long = df_can.melt(id_vars='period', var_name='state', value_name='daily_value')
    province_map = {
        'AB':'Alberta','BC':'British Columbia','MB':'Manitoba','NB':'New Brunswick',
        'NL':'Newfoundland & Labrador','NS':'Nova Scotia','ON':'Ontario','PE':'Prince Edward Island',
        'QC':'Quebec','SK':'Saskatchewan','NT':'Northwest Territories','NW':'Northwest Territories',
        'NU':'Nunavut','YT':'Yukon'
    }
    df_long['state'] = df_long['state'].map(province_map).fillna(df_long['state'])
    df_long['country'] = 'Canada'
    df_long['year'] = df_long['period'].dt.year
    df_long['month'] = df_long['period'].dt.month
    return df_long[df_long['year'] >= 2000]

In [15]:
df_oil_can = load_canadian_data()
df_oil_can.head()

,period,state,daily_value,country,year,month
0,2000-01-01,Newfoundland & Labrador,153.972985,Canada,2000,1
1,2000-02-01,Newfoundland & Labrador,141.894071,Canada,2000,2
2,2000-03-01,Newfoundland & Labrador,152.561027,Canada,2000,3
3,2000-04-01,Newfoundland & Labrador,140.933641,Canada,2000,4
4,2000-05-01,Newfoundland & Labrador,169.385917,Canada,2000,5


In [16]:
# Combine US and Canada Data
df_all = pd.concat([df_oil, df_oil_can], ignore_index=True)
# Exclude data after January 2025
df_all = df_all[df_all['period'] <= pd.Timestamp('2025-01-31')]

padd_to_states = {
    "PADD 1": ['Maine','New Hampshire','Vermont','Massachusetts','Rhode Island','Connecticut',
               'New York','New Jersey','Pennsylvania'],
    "PADD 2": ['Ohio','Michigan','Indiana','Illinois','Wisconsin','Minnesota','Iowa','Missouri',
               'North Dakota','South Dakota','Nebraska','Kansas'],
    "PADD 3": ['Texas','Louisiana','Mississippi','Alabama','Florida','Arkansas','Oklahoma',
               'New Mexico','Federal Offshore (Gulf)'],
    "PADD 4": ['Montana','Idaho','Wyoming','Colorado','Utah','Nevada','Arizona','Washington',
               'Oregon'],
    "PADD 5": ['California','Alaska','Hawaii','Washington (Pacific)','Oregon (Pacific)',
               'Federal Offshore (Pacific)']
}

countries = df_all['country'].unique()
states_by_country = {c: sorted(df_all[df_all['country']==c]['state'].unique()) for c in countries}

In [17]:
df_all

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units,country,state,year,month,days_in_month,daily_value,name,value_x,value_y
0,2000-01-01,SFL,FLORIDA,EPC0,Crude Oil,FPF,Field Production,MCRFPFL1,Florida Field Production of Crude Oil (Thousan...,360.0,MBBL,USA,Florida,2000,1,31.0,11.612903,NaN,NaN,NaN
1,2000-02-01,SFL,FLORIDA,EPC0,Crude Oil,FPF,Field Production,MCRFPFL1,Florida Field Production of Crude Oil (Thousan...,349.0,MBBL,USA,Florida,2000,2,29.0,12.034483,NaN,NaN,NaN
2,2000-03-01,SFL,FLORIDA,EPC0,Crude Oil,FPF,Field Production,MCRFPFL1,Florida Field Production of Crude Oil (Thousan...,373.0,MBBL,USA,Florida,2000,3,31.0,12.032258,NaN,NaN,NaN
3,2000-04-01,SFL,FLORIDA,EPC0,Crude Oil,FPF,Field Production,MCRFPFL1,Florida Field Production of Crude Oil (Thousan...,385.0,MBBL,USA,Florida,2000,4,30.0,12.833333,NaN,NaN,NaN
4,2000-05-01,SFL,FLORIDA,EPC0,Crude Oil,FPF,Field Production,MCRFPFL1,Florida Field Production of Crude Oil (Thousan...,411.0,MBBL,USA,Florida,2000,5,31.0,13.258065,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18838,2024-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Canada,Nova Scotia,2024,9,NaN,0.000000,NaN,NaN,NaN
18839,2024-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Canada,Nova Scotia,2024,10,NaN,0.000000,NaN,NaN,NaN
18840,2024-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Canada,Nova Scotia,2024,11,NaN,0.000000,NaN,NaN,NaN
18841,2024-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Canada,Nova Scotia,2024,12,NaN,0.000000,NaN,NaN,NaN


In [18]:
# Let's drop the columns that are not needed
df_all.drop(columns=['duoarea', 'area-name', 'product', 'product-name', 'process', 'process-name', 
                     'series', 'series-description', 'name', 'value_x', 'value_y'], inplace=True)

# Let's perform some aggregation
df_all['days_in_month'] = df_all['days_in_month'].fillna(df_all['period'].dt.days_in_month)
df_all['value'] = df_all['value'].fillna(df_all['daily_value'] * df_all['days_in_month'])

df_all

,period,value,units,country,state,year,month,days_in_month,daily_value
0,2000-01-01,360.0,MBBL,USA,Florida,2000,1,31.0,11.612903
1,2000-02-01,349.0,MBBL,USA,Florida,2000,2,29.0,12.034483
2,2000-03-01,373.0,MBBL,USA,Florida,2000,3,31.0,12.032258
3,2000-04-01,385.0,MBBL,USA,Florida,2000,4,30.0,12.833333
4,2000-05-01,411.0,MBBL,USA,Florida,2000,5,31.0,13.258065
...,...,...,...,...,...,...,...,...,...
18838,2024-09-01,0.0,NaN,Canada,Nova Scotia,2024,9,30.0,0.000000
18839,2024-10-01,0.0,NaN,Canada,Nova Scotia,2024,10,31.0,0.000000
18840,2024-11-01,0.0,NaN,Canada,Nova Scotia,2024,11,30.0,0.000000
18841,2024-12-01,0.0,NaN,Canada,Nova Scotia,2024,12,31.0,0.000000


### Map plots to visualize the production of each state/province

Here, we will recall the geospatial dataset for the USA and introduce that of Canada. Then we will unify it as one dataframe to facilitate the development of the geopandas maps. 

We'd start by renaming the name column of states_prov dataframe to state

In [19]:
can_prov = gpd.read_file('/Users/boluwatifeoduyemi/Documents/Data Science/Python Packages/georef_canada_province_public/georef-canada-province-millesime.shp')
can_prov.head()

,year,prov_code,prov_name_e,prov_area_c,prov_type,prov_name_f,geometry
0,2021,['12'],['Nova Scotia'],CAN,province,Nouvelle-Écosse,"MULTIPOLYGON (((-65.60612 43.51377, -65.55855 ..."
1,2021,['24'],['Quebec'],CAN,province,Québec,"MULTIPOLYGON (((-73.86407 45.51852, -73.91222 ..."
2,2021,['61'],['Northwest Territories'],CAN,territory / territoire,Territoires du Nord-Ouest,"MULTIPOLYGON (((-109.83296 75.93393, -109.8323..."
3,2021,['11'],['Prince Edward Island'],CAN,province,Île-du-Prince-Édouard,"POLYGON ((-61.98606 46.46286, -62.00568 46.429..."
4,2021,['46'],['Manitoba'],CAN,province,Manitoba,"POLYGON ((-94.82808 60, -94.85275 59.97128, -9..."


Similarly, we will rename the prov_name_e column to state and change the format from ['state_name] to state_name

In [20]:
# Remove square brackets and convert to plain strings
can_prov['prov_name_e'] = can_prov['prov_name_e'].str.strip("[]").str.replace("'", "").str.strip()
can_prov['prov_name_e'] = can_prov['prov_name_e'].replace('Newfoundland and Labrador', 'Newfoundland & Labrador')

can_prov.head()


,year,prov_code,prov_name_e,prov_area_c,prov_type,prov_name_f,geometry
0,2021,['12'],Nova Scotia,CAN,province,Nouvelle-Écosse,"MULTIPOLYGON (((-65.60612 43.51377, -65.55855 ..."
1,2021,['24'],Quebec,CAN,province,Québec,"MULTIPOLYGON (((-73.86407 45.51852, -73.91222 ..."
2,2021,['61'],Northwest Territories,CAN,territory / territoire,Territoires du Nord-Ouest,"MULTIPOLYGON (((-109.83296 75.93393, -109.8323..."
3,2021,['11'],Prince Edward Island,CAN,province,Île-du-Prince-Édouard,"POLYGON ((-61.98606 46.46286, -62.00568 46.429..."
4,2021,['46'],Manitoba,CAN,province,Manitoba,"POLYGON ((-94.82808 60, -94.85275 59.97128, -9..."


In [21]:
can_prov['prov_name_e'].unique()

array(['Nova Scotia', 'Quebec', 'Northwest Territories',
       'Prince Edward Island', 'Manitoba', 'Alberta', 'Saskatchewan',
       'Yukon', 'Newfoundland & Labrador', 'New Brunswick', 'Nunavut',
       'British Columbia', 'Ontario'], dtype=object)

In [22]:
# Some Canadian provinces and terrotories are missing in the df_all
# Let's identify the missing provinces and territories
missing_provinces_can = set(can_prov['prov_name_e'].unique()) - set(df_all['state'].unique())
print("Missing provinces and territories in df_all:", missing_provinces_can)

Missing provinces and territories in df_all: {'Quebec', 'Nunavut', 'Yukon', 'Prince Edward Island'}


In [23]:
# Similar to what was done for the US missing states, create a DataFrame with the first day of every month from 2000 to 2025
periods_can = pd.date_range(start="2000-01-01", end="2025-01-01", freq="MS")  # 'MS' means Month Start
all_periods_can = pd.DataFrame({
    'period': periods_can,
    'value': 0  # Assign 0 to all values
})

all_periods_can['period'] = pd.to_datetime(all_periods_can['period'], errors='coerce')

all_periods_can.head()

,period,value
0,2000-01-01,0
1,2000-02-01,0
2,2000-03-01,0
3,2000-04-01,0
4,2000-05-01,0


In [24]:
# Extract data for missing states from can_prov
missing_states_can = can_prov[can_prov['prov_name_e'].isin(missing_provinces_can)][['prov_name_e']].copy()

# Assign values of 0 to the 'value' and 'daily_value' column for missing states
missing_states_can["value"] = 0
missing_states_can["daily_value"] = 0

# Add a key column for cross join
missing_states_can['key'] = 1
all_periods_can['key'] = 1

# Perform cross join to get all combinations
missing_prov = pd.merge(missing_states_can, all_periods_can, on='key').drop('key', axis=1)
missing_prov

,prov_name_e,value_x,daily_value,period,value_y
0,Quebec,0,0,2000-01-01,0
1,Quebec,0,0,2000-02-01,0
2,Quebec,0,0,2000-03-01,0
3,Quebec,0,0,2000-04-01,0
4,Quebec,0,0,2000-05-01,0
...,...,...,...,...,...
1199,Nunavut,0,0,2024-09-01,0
1200,Nunavut,0,0,2024-10-01,0
1201,Nunavut,0,0,2024-11-01,0
1202,Nunavut,0,0,2024-12-01,0


In [25]:
# Add the missing provinces new data to df_all
df_all = pd.concat([df_all, missing_prov], ignore_index=True)

# Extract the year and month from the 'period' column
df_all['year'] = df_all['period'].dt.year
df_all['month'] = df_all['period'].dt.month

# Verify the updated df_oil
df_all

,period,value,units,country,state,year,month,days_in_month,daily_value,prov_name_e,value_x,value_y
0,2000-01-01,360.0,MBBL,USA,Florida,2000,1,31.0,11.612903,NaN,NaN,NaN
1,2000-02-01,349.0,MBBL,USA,Florida,2000,2,29.0,12.034483,NaN,NaN,NaN
2,2000-03-01,373.0,MBBL,USA,Florida,2000,3,31.0,12.032258,NaN,NaN,NaN
3,2000-04-01,385.0,MBBL,USA,Florida,2000,4,30.0,12.833333,NaN,NaN,NaN
4,2000-05-01,411.0,MBBL,USA,Florida,2000,5,31.0,13.258065,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
19861,2024-09-01,NaN,NaN,NaN,NaN,2024,9,NaN,0.000000,Nunavut,0.0,0.0
19862,2024-10-01,NaN,NaN,NaN,NaN,2024,10,NaN,0.000000,Nunavut,0.0,0.0
19863,2024-11-01,NaN,NaN,NaN,NaN,2024,11,NaN,0.000000,Nunavut,0.0,0.0
19864,2024-12-01,NaN,NaN,NaN,NaN,2024,12,NaN,0.000000,Nunavut,0.0,0.0


In [26]:
# Let's perform some aggregation
df_all['days_in_month'] = df_all['days_in_month'].fillna(df_all['period'].dt.days_in_month)
df_all['value'] = df_all['value'].fillna(df_all['daily_value'] * df_all['days_in_month'])

# Replace the NaN values in the 'state' column with the corresponding values from the 'prov_name_e' column
df_all['state'] = df_all['state'].fillna(df_all['prov_name_e'])

# Let's drop the columns that are not needed
df_all.drop(columns=['prov_name_e', 'value_x', 'value_y'], inplace=True)

# Replace NaN in the 'country' column with 'Canada' for specific states
df_all.loc[df_all['state'].isin(['Yukon', 'Prince Edward Island', 'Nunavut', 'Quebec']), 'country'] = df_all['country'].fillna('Canada')

df_all

,period,value,units,country,state,year,month,days_in_month,daily_value
0,2000-01-01,360.0,MBBL,USA,Florida,2000,1,31.0,11.612903
1,2000-02-01,349.0,MBBL,USA,Florida,2000,2,29.0,12.034483
2,2000-03-01,373.0,MBBL,USA,Florida,2000,3,31.0,12.032258
3,2000-04-01,385.0,MBBL,USA,Florida,2000,4,30.0,12.833333
4,2000-05-01,411.0,MBBL,USA,Florida,2000,5,31.0,13.258065
...,...,...,...,...,...,...,...,...,...
19861,2024-09-01,0.0,NaN,Canada,Nunavut,2024,9,30.0,0.000000
19862,2024-10-01,0.0,NaN,Canada,Nunavut,2024,10,31.0,0.000000
19863,2024-11-01,0.0,NaN,Canada,Nunavut,2024,11,30.0,0.000000
19864,2024-12-01,0.0,NaN,Canada,Nunavut,2024,12,31.0,0.000000


In [27]:
df_all['country'].unique()

array(['USA', 'Canada'], dtype=object)

In [28]:
# Standardize the 'state' column in df_all
df_all['state'] = df_all['state'].str.strip()

# Standardize the 'name' column in states_prov
states_prov['name'] = states_prov['name'].str.strip()

In [29]:
# Merge df_all with states_prov based on the common keys
maps_oil = df_all.merge(states_prov[['name', 'iso_3166_2', 'geometry']], 
                         how='left',
                         left_on='state', 
                         right_on='name')

maps_oil.head()

,period,value,units,country,state,year,month,days_in_month,daily_value,name,iso_3166_2,geometry
0,2000-01-01,360.0,MBBL,USA,Florida,2000,1,31.0,11.612903,Florida,US-FL,"POLYGON ((-87.53039 30.2742, -87.45789 30.4112..."
1,2000-02-01,349.0,MBBL,USA,Florida,2000,2,29.0,12.034483,Florida,US-FL,"POLYGON ((-87.53039 30.2742, -87.45789 30.4112..."
2,2000-03-01,373.0,MBBL,USA,Florida,2000,3,31.0,12.032258,Florida,US-FL,"POLYGON ((-87.53039 30.2742, -87.45789 30.4112..."
3,2000-04-01,385.0,MBBL,USA,Florida,2000,4,30.0,12.833333,Florida,US-FL,"POLYGON ((-87.53039 30.2742, -87.45789 30.4112..."
4,2000-05-01,411.0,MBBL,USA,Florida,2000,5,31.0,13.258065,Florida,US-FL,"POLYGON ((-87.53039 30.2742, -87.45789 30.4112..."


In [30]:
# Merge maps_oil with can_prov to get missing geometry values
maps_oil_update = maps_oil.merge(
    can_prov[['prov_name_e', 'geometry']],
    how='left',
    left_on='state',
    right_on='prov_name_e',
    suffixes=('', '_can')  # prevents column name conflict
)

# Update geometry where it is missing in maps_oil
maps_oil_update['geometry'] = maps_oil_update['geometry'].combine_first(maps_oil_update['geometry_can'])

# Step 3: Drop temporary helper columns
maps_oil_update.drop(columns=['prov_name_e', 'geometry_can'], inplace=True)

# Final updated DataFrame
maps_oil = maps_oil_update

# Preview result
maps_oil.head()


,period,value,units,country,state,year,month,days_in_month,daily_value,name,iso_3166_2,geometry
0,2000-01-01,360.0,MBBL,USA,Florida,2000,1,31.0,11.612903,Florida,US-FL,"POLYGON ((-87.53039 30.2742, -87.45789 30.4112..."
1,2000-02-01,349.0,MBBL,USA,Florida,2000,2,29.0,12.034483,Florida,US-FL,"POLYGON ((-87.53039 30.2742, -87.45789 30.4112..."
2,2000-03-01,373.0,MBBL,USA,Florida,2000,3,31.0,12.032258,Florida,US-FL,"POLYGON ((-87.53039 30.2742, -87.45789 30.4112..."
3,2000-04-01,385.0,MBBL,USA,Florida,2000,4,30.0,12.833333,Florida,US-FL,"POLYGON ((-87.53039 30.2742, -87.45789 30.4112..."
4,2000-05-01,411.0,MBBL,USA,Florida,2000,5,31.0,13.258065,Florida,US-FL,"POLYGON ((-87.53039 30.2742, -87.45789 30.4112..."


In [31]:
maps_oil.isna().sum()

period              0
value               0
units            9933
country             0
state               0
year                0
month               0
days_in_month       0
daily_value         0
name             4515
iso_3166_2       4515
geometry          602
dtype: int64

In [32]:
# Find rows where geometry is missing
missing_geometry_states = maps_oil[maps_oil['geometry'].isna()]['state'].unique()

# Display the unique states missing geometry
print("Unique states missing geometry:", missing_geometry_states)

Unique states missing geometry: ['Federal Offshore (PADD 3)' 'Federal Offshore (PADD 5)']


Notice that Federal Offshore (PADD 3 and PADD 5) do not have the geometry data. Because the geospatial data does not contain PADDs. Due to this we will have to remove the PADDs from the dataset. 

In [33]:
# Drop rows where 'state' is 'Federal Offshore (PADD 3)' or 'Federal Offshore (PADD 5)'
maps_oil = maps_oil[~maps_oil['state'].isin(['Federal Offshore (PADD 3)', 'Federal Offshore (PADD 5)'])]

maps_oil['state'].unique()

array(['Florida', 'New York', 'Pennsylvania', 'Virginia', 'West Virginia',
       'Illinois', 'Indiana', 'Kansas', 'Kentucky', 'Michigan',
       'Missouri', 'Nebraska', 'North Dakota', 'Ohio', 'Oklahoma',
       'South Dakota', 'Tennessee', 'Alabama', 'Arkansas', 'Louisiana',
       'Mississippi', 'New Mexico', 'Texas', 'Colorado', 'Montana',
       'Utah', 'Wyoming', 'Alaska', 'Arizona', 'California', 'Nevada',
       'Minnesota', 'Hawaii', 'Idaho', 'Washington', 'Oregon', 'Iowa',
       'Connecticut', 'Massachusetts', 'New Hampshire', 'Rhode Island',
       'Vermont', 'Georgia', 'South Carolina', 'North Carolina',
       'Wisconsin', 'Delaware', 'District of Columbia', 'Maryland',
       'New Jersey', 'Maine', 'Newfoundland & Labrador', 'Ontario',
       'New Brunswick', 'Manitoba', 'Northwest Territories',
       'Saskatchewan', 'Alberta', 'British Columbia', 'Nova Scotia',
       'Quebec', 'Prince Edward Island', 'Yukon', 'Nunavut'], dtype=object)

In [34]:
# Assign the appropriate values to iso_3166_2
# Define a mapping of state names to their abbreviations
state_abbreviations = {
    'Newfoundland & Labrador': 'NL',
    'Ontario': 'ON',
    'New Brunswick': 'NB',
    'Manitoba': 'MB',
    'Northwest Territories': 'NT',
    'British Columbia': 'BC',
    'Nova Scotia': 'NS',
    'Alberta': 'AB',
    'Saskatchewan': 'SK',
    'Yukon': 'YT',
    'Quebec': 'QC',
    'Prince Edward Island': 'PE',
    'Nunavut': 'NU'
}

# Replace NaN cells in 'iso_3166_2' with 'CA-{state abbreviation}'
maps_oil['iso_3166_2'] = maps_oil.apply(
    lambda row: f"CA-{state_abbreviations[row['state']]}" if pd.isna(row['iso_3166_2']) and row['state'] in state_abbreviations else row['iso_3166_2'],
    axis=1
)

maps_oil

/var/folders/h2/bnhyxdfd2cb599q0cgnmcxrr0000gn/T/ipykernel_62702/3529580063.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  maps_oil['iso_3166_2'] = maps_oil.apply(


,period,value,units,country,state,year,month,days_in_month,daily_value,name,iso_3166_2,geometry
0,2000-01-01,360.0,MBBL,USA,Florida,2000,1,31.0,11.612903,Florida,US-FL,"POLYGON ((-87.53039 30.2742, -87.45789 30.4112..."
1,2000-02-01,349.0,MBBL,USA,Florida,2000,2,29.0,12.034483,Florida,US-FL,"POLYGON ((-87.53039 30.2742, -87.45789 30.4112..."
2,2000-03-01,373.0,MBBL,USA,Florida,2000,3,31.0,12.032258,Florida,US-FL,"POLYGON ((-87.53039 30.2742, -87.45789 30.4112..."
3,2000-04-01,385.0,MBBL,USA,Florida,2000,4,30.0,12.833333,Florida,US-FL,"POLYGON ((-87.53039 30.2742, -87.45789 30.4112..."
4,2000-05-01,411.0,MBBL,USA,Florida,2000,5,31.0,13.258065,Florida,US-FL,"POLYGON ((-87.53039 30.2742, -87.45789 30.4112..."
...,...,...,...,...,...,...,...,...,...,...,...,...
19861,2024-09-01,0.0,NaN,Canada,Nunavut,2024,9,30.0,0.000000,NaN,CA-NU,"MULTIPOLYGON (((-106.67059 73.67679, -106.4866..."
19862,2024-10-01,0.0,NaN,Canada,Nunavut,2024,10,31.0,0.000000,NaN,CA-NU,"MULTIPOLYGON (((-106.67059 73.67679, -106.4866..."
19863,2024-11-01,0.0,NaN,Canada,Nunavut,2024,11,30.0,0.000000,NaN,CA-NU,"MULTIPOLYGON (((-106.67059 73.67679, -106.4866..."
19864,2024-12-01,0.0,NaN,Canada,Nunavut,2024,12,31.0,0.000000,NaN,CA-NU,"MULTIPOLYGON (((-106.67059 73.67679, -106.4866..."


In [35]:
# Drop the 'name' column as it's no longer needed
maps_oil.drop(columns=['name'], inplace=True)

maps_oil

/var/folders/h2/bnhyxdfd2cb599q0cgnmcxrr0000gn/T/ipykernel_62702/459133084.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  maps_oil.drop(columns=['name'], inplace=True)


,period,value,units,country,state,year,month,days_in_month,daily_value,iso_3166_2,geometry
0,2000-01-01,360.0,MBBL,USA,Florida,2000,1,31.0,11.612903,US-FL,"POLYGON ((-87.53039 30.2742, -87.45789 30.4112..."
1,2000-02-01,349.0,MBBL,USA,Florida,2000,2,29.0,12.034483,US-FL,"POLYGON ((-87.53039 30.2742, -87.45789 30.4112..."
2,2000-03-01,373.0,MBBL,USA,Florida,2000,3,31.0,12.032258,US-FL,"POLYGON ((-87.53039 30.2742, -87.45789 30.4112..."
3,2000-04-01,385.0,MBBL,USA,Florida,2000,4,30.0,12.833333,US-FL,"POLYGON ((-87.53039 30.2742, -87.45789 30.4112..."
4,2000-05-01,411.0,MBBL,USA,Florida,2000,5,31.0,13.258065,US-FL,"POLYGON ((-87.53039 30.2742, -87.45789 30.4112..."
...,...,...,...,...,...,...,...,...,...,...,...
19861,2024-09-01,0.0,NaN,Canada,Nunavut,2024,9,30.0,0.000000,CA-NU,"MULTIPOLYGON (((-106.67059 73.67679, -106.4866..."
19862,2024-10-01,0.0,NaN,Canada,Nunavut,2024,10,31.0,0.000000,CA-NU,"MULTIPOLYGON (((-106.67059 73.67679, -106.4866..."
19863,2024-11-01,0.0,NaN,Canada,Nunavut,2024,11,30.0,0.000000,CA-NU,"MULTIPOLYGON (((-106.67059 73.67679, -106.4866..."
19864,2024-12-01,0.0,NaN,Canada,Nunavut,2024,12,31.0,0.000000,CA-NU,"MULTIPOLYGON (((-106.67059 73.67679, -106.4866..."


In [36]:
# Let's convert maps_oil to a GeoDataFrame
geo_maps_oil = gpd.GeoDataFrame(maps_oil, geometry='geometry')
geo_maps_oil.head()

,period,value,units,country,state,year,month,days_in_month,daily_value,iso_3166_2,geometry
0,2000-01-01,360.0,MBBL,USA,Florida,2000,1,31.0,11.612903,US-FL,"POLYGON ((-87.53039 30.2742, -87.45789 30.4112..."
1,2000-02-01,349.0,MBBL,USA,Florida,2000,2,29.0,12.034483,US-FL,"POLYGON ((-87.53039 30.2742, -87.45789 30.4112..."
2,2000-03-01,373.0,MBBL,USA,Florida,2000,3,31.0,12.032258,US-FL,"POLYGON ((-87.53039 30.2742, -87.45789 30.4112..."
3,2000-04-01,385.0,MBBL,USA,Florida,2000,4,30.0,12.833333,US-FL,"POLYGON ((-87.53039 30.2742, -87.45789 30.4112..."
4,2000-05-01,411.0,MBBL,USA,Florida,2000,5,31.0,13.258065,US-FL,"POLYGON ((-87.53039 30.2742, -87.45789 30.4112..."


In [37]:
# Let's check the data types
geo_maps_oil.dtypes

period           datetime64[ns]
value                   float64
units                    object
country                  object
state                    object
year                      int32
month                     int32
days_in_month           float64
daily_value             float64
iso_3166_2               object
geometry               geometry
dtype: object

In [38]:
# To create an interactive map, we need to ensure that the period column is in datetime format
geo_maps_oil['period'] = pd.to_datetime(geo_maps_oil['period'])

# Afterward, extract the month from the 'period' column
geo_maps_oil['month_name'] = geo_maps_oil['period'].dt.month_name()

# Reorder the columns to place 'month_name' next to 'month'
columns = list(geo_maps_oil.columns)
month_index = columns.index('month')  # Find the index of the 'month' column
columns.insert(month_index + 1, columns.pop(columns.index('month_name')))  # Move 'month_name' next to 'month'
geo_maps_oil = geo_maps_oil[columns]

geo_maps_oil

,period,value,units,country,state,year,month,month_name,days_in_month,daily_value,iso_3166_2,geometry
0,2000-01-01,360.0,MBBL,USA,Florida,2000,1,January,31.0,11.612903,US-FL,"POLYGON ((-87.53039 30.2742, -87.45789 30.4112..."
1,2000-02-01,349.0,MBBL,USA,Florida,2000,2,February,29.0,12.034483,US-FL,"POLYGON ((-87.53039 30.2742, -87.45789 30.4112..."
2,2000-03-01,373.0,MBBL,USA,Florida,2000,3,March,31.0,12.032258,US-FL,"POLYGON ((-87.53039 30.2742, -87.45789 30.4112..."
3,2000-04-01,385.0,MBBL,USA,Florida,2000,4,April,30.0,12.833333,US-FL,"POLYGON ((-87.53039 30.2742, -87.45789 30.4112..."
4,2000-05-01,411.0,MBBL,USA,Florida,2000,5,May,31.0,13.258065,US-FL,"POLYGON ((-87.53039 30.2742, -87.45789 30.4112..."
...,...,...,...,...,...,...,...,...,...,...,...,...
19861,2024-09-01,0.0,NaN,Canada,Nunavut,2024,9,September,30.0,0.000000,CA-NU,"MULTIPOLYGON (((-106.67059 73.67679, -106.4866..."
19862,2024-10-01,0.0,NaN,Canada,Nunavut,2024,10,October,31.0,0.000000,CA-NU,"MULTIPOLYGON (((-106.67059 73.67679, -106.4866..."
19863,2024-11-01,0.0,NaN,Canada,Nunavut,2024,11,November,30.0,0.000000,CA-NU,"MULTIPOLYGON (((-106.67059 73.67679, -106.4866..."
19864,2024-12-01,0.0,NaN,Canada,Nunavut,2024,12,December,31.0,0.000000,CA-NU,"MULTIPOLYGON (((-106.67059 73.67679, -106.4866..."


In [ ]:
import plotly.express as px
from ipywidgets import interact, Dropdown
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.graph_objects as go
from shapely.geometry import mapping

In [47]:
# --------------------------
# 1. Prepare Filter Widgets
# --------------------------

# Dropdowns
start_year_dropdown = widgets.Dropdown(
    options=sorted(geo_maps_oil['year'].dropna().unique()),
    value=geo_maps_oil['year'].min(),
    description='Start Year'
)

end_year_dropdown = widgets.Dropdown(
    options=sorted(geo_maps_oil['year'].dropna().unique()),
    value=geo_maps_oil['year'].max(),
    description='End Year'
)

start_month_dropdown = widgets.Dropdown(
    options=sorted(geo_maps_oil['month_name'].dropna().unique(), key=lambda m: pd.to_datetime(m, format='%B').month),
    value='January',
    description='Start Month'
)

end_month_dropdown = widgets.Dropdown(
    options=sorted(geo_maps_oil['month_name'].dropna().unique(), key=lambda m: pd.to_datetime(m, format='%B').month),
    value='December',
    description='End Month'
)

state_dropdown = widgets.Dropdown(
    options=['All'] + sorted(geo_maps_oil['state'].dropna().unique()),
    value='All',
    description='State'
)

filter_button = widgets.Button(description="Update Map", button_style='success')

# Output placeholder for the map
map_output = widgets.Output()

# --------------------------------
# 2. Define Filtering + Map Logic
# --------------------------------

def update_map(change=None):
    with map_output:
        clear_output()

        # Filter the data
        filtered = geo_maps_oil.copy()

        # Convert month names to numbers for comparison
        month_order = {month: idx for idx, month in enumerate(pd.date_range('2000-01-01', periods=12, freq='MS').strftime('%B'), start=1)}
        start_month_num = month_order[start_month_dropdown.value]
        end_month_num = month_order[end_month_dropdown.value]

        filtered = filtered[
            (filtered['year'] >= start_year_dropdown.value) &
            (filtered['year'] <= end_year_dropdown.value) &
            (filtered['month'].between(start_month_num, end_month_num))
        ]

        if state_dropdown.value != 'All':
            filtered = filtered[filtered['state'] == state_dropdown.value]

        if filtered.empty:
            print("No data available for selected filters.")
            return

        # Calculate number of months in range
        start_date = pd.to_datetime(f"{start_year_dropdown.value}-{start_month_num:02d}-01")
        end_date = pd.to_datetime(f"{end_year_dropdown.value}-{end_month_num:02d}-01")
        num_months = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month) + 1

        # Aggregate filtered data and calculate average
        average_data = filtered.groupby(['state'], as_index=False).agg({
            'daily_value': 'sum'
        })
        average_data['daily_value'] = average_data['daily_value'] / num_months

        # Build GeoJSON from unique geometries
        features = []
        for _, row in filtered.drop_duplicates('state')[['state', 'geometry']].iterrows():
            if pd.notnull(row['geometry']):
                features.append({
                    'type': 'Feature',
                    'id': row['state'],
                    'properties': {},
                    'geometry': mapping(row['geometry'])
                })

        geojson = {
            'type': 'FeatureCollection',
            'features': features
        }

        # Plot with Plotly
        fig = px.choropleth(
            average_data,
            geojson=geojson,
            locations='state',
            color='daily_value',
            hover_name='state',
            title='Average Daily Oil Production by State',
            color_continuous_scale='plasma',
            labels={'daily_value': 'Average Daily Production (MBBL)'}
        )

        fig.update_geos(fitbounds="locations", visible=False)
        fig.update_layout(margin={"r":0,"t":50,"l":0,"b":0})

        display(fig)

# Trigger update when button is clicked
filter_button.on_click(update_map)

# ----------------------------------
# 3. Display UI and Initial Map
# ----------------------------------

# Arrange layout
ui = widgets.VBox([
    widgets.HBox([start_year_dropdown, end_year_dropdown]),
    widgets.HBox([start_month_dropdown, end_month_dropdown]),
    widgets.HBox([state_dropdown]),
    filter_button,
    map_output
])

# Show filter UI and initial map
display(ui)
update_map()


In [48]:
geo_maps_oil.to_csv('geo_maps_oil_UPDATED.csv', index=False)